In [1]:
import os
import datetime
import argparse
import pandas as pd
import numpy as np
import glob
import pytz
from sklearn.preprocessing import StandardScaler
from gauss_rank import GaussRankScaler
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = '/home/gmoreira/dataset/gcom_news'

In [3]:
OUTPUT_PATH = '/home/gmoreira/dataset/gcom_news/clicks_preprocessed'
!mkdir -p $OUTPUT_PATH

In [4]:
def extract_local_hour_weekday(timestamp_in_utc, local_tz):
    dt = pytz.utc.localize(datetime.datetime.utcfromtimestamp(timestamp_in_utc)).astimezone(pytz.timezone(local_tz))
    return dt.hour + (dt.minute/60.0), dt.weekday() #First day is Monday

In [5]:
def get_cicled_feature_value(value, max_value):
    value_scaled = (value + 0.000001) / max_value
    value_sin = np.sin(2*np.pi*value_scaled)
    value_cos = np.cos(2*np.pi*value_scaled)
    return value_sin, value_cos

def get_time_features(timestamp, default_timezone='America/Sao_Paulo'):
    #Converting timestamp to the fDefault timezone, where most clicks originate
    local_hour, local_weekday = extract_local_hour_weekday(int(timestamp)//1000, default_timezone)        
    #Converting hour in two cycling features to represent the continuity of the hours
    local_hour_sin, local_hour_cos = get_cicled_feature_value(local_hour, 24)
    local_weekday_sin, local_weekday_cos = get_cicled_feature_value(local_weekday+1, 7)
    return local_hour_sin, local_hour_cos, local_weekday_sin, local_weekday_cos

In [6]:
def group_sessions(clicks_hour_df):
    def to_list(series):
        return list(series)
    
    #Ensuring that sessions are chronologically ordered
    clicks_hour_df.sort_values(['session_start', 'click_timestamp'], inplace=True)
    sessions_by_hour_df = clicks_hour_df.groupby('session_id').agg({'user_id': min,
                                                                    'session_start': min,
                                                                    'session_size': min,
                                                                    'click_article_id': to_list,
                                                                    'click_timestamp': to_list,
                                                                    'click_environment': to_list,
                                                                    'click_deviceGroup': to_list,
                                                                    'click_os': to_list,
                                                                    'click_country': to_list,
                                                                    'click_region': to_list,
                                                                    'click_referrer_type': to_list,
                                                                    'hour_sin': to_list,
                                                                    'hour_cos': to_list,
                                                                    'weekday_sin': to_list,
                                                                    'weekday_cos': to_list,
                                                                    'item_age_hours': to_list,
                                                                    'item_age_hours_norm': to_list,
                                                                    }
                                                                   ).reset_index()
    return sessions_by_hour_df

In [7]:
def preprocess_interactions(clicks_df):    
    clicks_df = clicks_df.merge(articles_metadata_df, 
                                          left_on='click_article_id', right_on='article_id')
    clicks_df[['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']] = \
            list(clicks_df['click_timestamp'].apply(lambda ts: get_time_features(ts)).values)
    
    clicks_df['item_age_hours'] = (clicks_df['click_timestamp'] - clicks_df['created_at_ts']) / (1000*60*60)
    
    scaler = GaussRankScaler()
    clicks_df['item_age_hours_norm'] = scaler.fit_transform(clicks_df[['item_age_hours']])
    return clicks_df

In [8]:
def prepare_sessions(clicks_hour_df):    
    sessions_df = group_sessions(clicks_hour_df)
    return sessions_df

In [9]:
#clicks_hour_df['item_age_hours'].describe(percentiles=np.arange(0.0,1.0,0.1))
'''
count     1883.000000
mean       313.920239
std       2500.257279
min        -20.069912
0%         -20.069912
10%          5.777035
20%          6.705644
30%          7.739401
40%          9.653872
50%         11.343312
60%         12.060842
70%         12.728053
80%         17.131041
90%         19.860367
max      33897.512268
Name: item_age_hours, dtype: float64
'''

'\ncount     1883.000000\nmean       313.920239\nstd       2500.257279\nmin        -20.069912\n0%         -20.069912\n10%          5.777035\n20%          6.705644\n30%          7.739401\n40%          9.653872\n50%         11.343312\n60%         12.060842\n70%         12.728053\n80%         17.131041\n90%         19.860367\nmax      33897.512268\nName: item_age_hours, dtype: float64\n'

In [10]:
articles_metadata_df = pd.read_csv(os.path.join(DATA_PATH, 'articles_metadata.csv'))

In [11]:
clicks_dfs_list = []
for idx, clicks_file_path in enumerate(sorted(glob.glob(f"{DATA_PATH}/clicks/*"))): 
    clicks_hour_df = pd.read_csv(clicks_file_path)
    clicks_hour_df['hour_index'] = idx
    clicks_dfs_list.append(clicks_hour_df)

In [12]:
clicks_merged_df = pd.concat(clicks_dfs_list, axis=0)
len(clicks_merged_df)

2988181

In [13]:
clicks_merged_df = preprocess_interactions(clicks_merged_df)
len(clicks_merged_df)

2988181

In [14]:
#Cardinalities of categoricals
max_series = clicks_merged_df.max()
max_series.to_dict()

{'user_id': 322896.0,
 'session_id': 1508211379189330.0,
 'session_start': 1508211379000.0,
 'session_size': 124.0,
 'click_article_id': 364046.0,
 'click_timestamp': 1510603454886.0,
 'click_environment': 4.0,
 'click_deviceGroup': 5.0,
 'click_os': 20.0,
 'click_country': 11.0,
 'click_region': 28.0,
 'click_referrer_type': 7.0,
 'hour_index': 384.0,
 'article_id': 364046.0,
 'category_id': 460.0,
 'created_at_ts': 1510666014000.0,
 'publisher_id': 0.0,
 'words_count': 6690.0,
 'hour_sin': 0.9999999999999657,
 'hour_cos': 0.9999999999999657,
 'weekday_sin': 0.9749277124471076,
 'weekday_cos': 0.9999999999995972,
 'item_age_hours': 94578.73542833333,
 'item_age_hours_norm': 2.75106390571208}

In [15]:
hour_index_min, hour_index_max = clicks_merged_df['hour_index'].min(), clicks_merged_df['hour_index'].max()
print(hour_index_min, hour_index_max)

0 384


In [16]:
counter = 1
for hour_index in range(hour_index_min, hour_index_max+1):
    clicks_hour_df = clicks_merged_df[clicks_merged_df['hour_index'] == hour_index]
    sessions_df = prepare_sessions(clicks_hour_df)
    
    if len(sessions_df) < 100:
        print("Ignoring this hour file, because has only {} sessions".format(len(sessions_df)))
        continue
    
    print(f"{hour_index}->{counter}", "# sessions: {}".format(len(sessions_df)))
    
    hour_files_path = os.path.join(OUTPUT_PATH, f"{counter:04}")
    os.makedirs(hour_files_path, exist_ok=True)
    
    #As the number of sessions by hour is small, uses all available sessions for training, 
    #and splits the same data as valid and test, as the evaluation is always performed in the next hour
    train_df = sessions_df
    sessions_df.to_parquet(os.path.join(hour_files_path, "train.parquet"))    
    valid_df, test_df = train_test_split(train_df, test_size=0.5, shuffle=True)
    valid_df.sort_values(['session_start']).to_parquet(os.path.join(hour_files_path, "valid.parquet"))    
    test_df.sort_values(['session_start']).to_parquet(os.path.join(hour_files_path, "test.parquet")) 
    
    counter += 1

/home/gmoreira/miniconda3/envs/transf4rec/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0->1 # sessions: 707


/home/gmoreira/miniconda3/envs/transf4rec/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


1->2 # sessions: 531
2->3 # sessions: 342


/home/gmoreira/miniconda3/envs/transf4rec/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/gmoreira/miniconda3/envs/transf4rec/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


...

382->379 # sessions: 2292


/home/gmoreira/miniconda3/envs/transf4rec/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


383->380 # sessions: 1397


/home/gmoreira/miniconda3/envs/transf4rec/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


384->381 # sessions: 954


In [63]:
test_df

,session_id,user_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type,hour_sin,hour_cos,weekday_sin,weekday_cos,item_age_hours,item_age_hours_norm
105,1508208072340482,112066,1508208072000,3,"[211442, 30760, 156279]","[1508208225821, 1508208328895, 1508208358895]","[4, 4, 4]","[3, 3, 3]","[2, 2, 2]","[1, 1, 1]","[12, 12, 12]","[2, 2, 2]","[0.1865242932136409, 0.19509057878510752, 0.19...","[0.9824503488935976, 0.98078522932867, 0.98078...","[0.9749277124471076, 0.9749277124471076, 0.974...","[-0.2225218090494724, -0.2225218090494724, -0....","[5.6960613888888885, 6.454970833333333, 8.4980...","[-1.2240418703593219, -0.6578974643947907, 0.0..."
921,1508211215779298,236837,1508211215000,5,"[209122, 70986, 205958, 224730, 211442]","[1508211237970, 1508211672092, 1508211823335, ...","[4, 4, 4, 4, 4]","[3, 3, 3, 3, 3]","[2, 2, 2, 2, 2]","[1, 1, 1, 1, 1]","[25, 25, 25, 25, 25]","[2, 2, 2, 2, 2]","[0.39474409692323004, 0.426568976687247, 0.434...","[0.9187911068051668, 0.9044550337789021, 0.900...","[0.9749277124471076, 0.9749277124471076, 0.974...","[-0.2225218090494724, -0.2225218090494724, -0....","[7.333602777777778, 7.3811366666666665, 7.2523...","[-0.27960043222329883, -0.26029603937469453, -..."
329,1508208786366706,148103,1508208786000,3,"[87215, 162378, 87181]","[1508209270702, 1508209380920, 1508209410920]","[4, 4, 4]","[3, 3, 3]","[2, 2, 2]","[1, 1, 1]","[25, 25, 25]","[2, 2, 2]","[0.26303146703869645, 0.2714407018352566, 0.27...","[0.9647872549673691, 0.9624551653906707, 0.962...","[0.9749277124471076, 0.9749277124471076, 0.974...","[-0.2225218090494724, -0.2225218090494724, -0....","[29.48991722222222, 32.7897, 28.610255555555554]","[0.928875640042075, 0.9522829694264864, 0.9239..."
741,1508210390659118,172415,1508210390000,3,"[285877, 285675, 16346]","[1508212188499, 1508213955178, 1508213985178]","[4, 4, 4]","[1, 1, 1]","[17, 17, 17]","[1, 1, 1]","[25, 25, 25]","[1, 1, 1]","[0.45787414786108444, 0.5699969777029488, 0.56...","[0.8890170216145953, 0.821646788717332, 0.8216...","[0.9749277124471076, 0.9749277124471076, 0.974...","[-0.2225218090494724, -0.2225218090494724, -0....","[14.73763861111111, 83.98699388888889, 8.93143...","[0.5490868212927918, 1.154554095885605, 0.1447..."
460,1508209319273837,41482,1508209319000,3,"[50644, 224730, 157815]","[1508209470352, 1508209713722, 1508209743722]","[4, 4, 4]","[4, 4, 4]","[20, 20, 20]","[1, 1, 1]","[25, 25, 25]","[7, 7, 7]","[0.2756376074747132, 0.29237195508272634, 0.29...","[0.961261623776595, 0.9563046794202694, 0.9550...","[0.9749277124471076, 0.9749277124471076, 0.974...","[-0.2225218090494724, -0.2225218090494724, -0....","[7.542042222222222, 8.002978333333333, 39.6213...","[-0.21934704387104617, -0.08562588730005356, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,1508210251402087,61735,1508210251000,2,"[209122, 234481]","[1508210263528, 1508210293528]","[4, 4]","[3, 3]","[20, 20]","[1, 1]","[25, 25]","[2, 2]","[0.3296908924247035, 0.33380710601672475]","[0.9440889340799428, 0.9426414037017147]","[0.9749277124471076, 0.9749277124471076]","[-0.2225218090494724, -0.2225218090494724]","[7.062924444444445, 7.978202222222222]","[-0.39171819088220244, -0.09327997209511539]"
240,1508208475329617,237783,1508208475000,2,"[70758, 70429]","[1508208506741, 1508208536741]","[2, 2]","[3, 3]","[20, 20]","[1, 1]","[21, 21]","[2, 2]","[0.20791194689619527, 0.20791194689619527]","[0.9781475463026187, 0.9781475463026187]","[0.9749277124471076, 0.9749277124471076]","[-0.2225218090494724, -0.2225218090494724]","[5.650761388888889, 13.720205833333333]","[-1.2732509596921786, 0.5151851032391899]"
339,1508208821279716,116647,1508208821000,3,"[144998, 205824, 142336]","[1508208960865, 1508209053958, 1508209083958]","[4, 4, 4]","[3, 3, 3]","[2, 2, 2]","[1, 1, 1]","[16, 16, 16]","[1, 1, 1]","[0.24192214962248654, 0.24615354677304335, 0.2...","[0.970295662

In [65]:
test_df.columns

Index(['session_id', 'user_id', 'session_start', 'session_size',
       'click_article_id', 'click_timestamp', 'click_environment',
       'click_deviceGroup', 'click_os', 'click_country', 'click_region',
       'click_referrer_type', 'hour_sin', 'hour_cos', 'weekday_sin',
       'weekday_cos', 'item_age_hours', 'item_age_hours_norm'],
      dtype='object')

In [19]:
train_df = pd.read_parquet('/home/gmoreira/dataset/gcom_news/clicks_preprocessed/0001/train.parquet')
len(train_df)

707

In [20]:
valid_df = pd.read_parquet('/home/gmoreira/dataset/gcom_news/clicks_preprocessed/0001/valid.parquet')
len(valid_df)

353

In [21]:
test_df = pd.read_parquet('/home/gmoreira/dataset/gcom_news/clicks_preprocessed/0001/test.parquet')
len(test_df)

354

In [26]:
assert not (valid_df['session_id'].isin(test_df['session_id'])).any()

In [27]:
train_df

,session_id,user_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type,hour_sin,hour_cos,weekday_sin,weekday_cos,item_age_hours,item_age_hours_norm
0,1506825423271737,0,1506825423000,2,"[157541, 68866]","[1506826828020, 1506826858020]","[4, 4]","[3, 3]","[20, 20]","[1, 1]","[20, 20]","[2, 2]","[2.6179938779914644e-07, 2.6179938779914644e-07]","[0.9999999999999657, 0.9999999999999657]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[7.308338888888889, 2.8822277777777776]","[-0.09762088770601476, -1.7279320974241994]"
1,1506825426267738,1,1506825426000,2,"[235840, 96663]","[1506827017951, 1506827047951]","[4, 4]","[1, 1]","[17, 17]","[1, 1]","[16, 16]","[2, 2]","[0.013089857348302801, 0.017452668196797395]","[0.9999143241471247, 0.9998476905873277]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[5.327486388888889, 10.839708611111112]","[-0.46633683105016777, 0.23392876152569825]"
2,1506825435299739,2,1506825435000,2,"[119592, 30970]","[1506827090575, 1506827120575]","[4, 4]","[1, 1]","[17, 17]","[1, 1]","[24, 24]","[2, 2]","[0.017452668196797395, 0.021815146771647007]","[0.9998476905873277, 0.9997620213687513]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[11.88182638888889, 14.868215277777777]","[0.29486848565302565, 0.44230751045686933]"
3,1506825442704740,3,1506825442000,2,"[236065, 236294]","[1506827536942, 1506827566942]","[4, 4]","[3, 3]","[2, 2]","[1, 1]","[21, 21]","[1, 1]","[0.05233621768354288, 0.05233621768354288]","[0.9986295210530184, 0.9986295210530184]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[35.61053944444444, 11.539983888888889]","[1.1211925734386485, 0.2755469292225951]"
4,1506825528135741,4,1506825528000,2,"[48915, 44488]","[1506826927593, 1506826957593]","[4, 4]","[1, 1]","[17, 17]","[1, 1]","[17, 17]","[1, 1]","[0.008726797287792916, 0.008726797287792916]","[0.9999619207795354, 0.9999619207795354]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[277.4062758333333, 18.037386944444446]","[1.4243143577529878, 0.5737429888900766]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,1506828951322439,702,1506828951000,2,"[235840, 284847]","[1506829061037, 1506829091037]","[2, 2]","[3, 3]","[20, 20]","[1, 1]","[21, 21]","[2, 2]","[0.1607428239988651, 0.16504786406963864]","[0.9869963244778928, 0.9862855583278355]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[5.895010277777778, 7.505010277777778]","[-0.3420879965175333, -0.0705984223772129]"
703,1506828958944440,703,1506828958000,2,"[84094, 83472]","[1506829062893, 1506829092893]","[4, 4]","[3, 3]","[2, 2]","[1, 1]","[7, 7]","[1, 1]","[0.1607428239988651, 0.16504786406963864]","[0.9869963244778928, 0.9862855583278355]","[8.975979006501141e-07, 8.975979006501141e-07]","[0.9999999999995972, 0.9999999999995972]","[19.27608138888889, 17.7546925]","[0.6220256059245568, 0.562620990669032]"
704,1506828960113441,704,1506828960000,5,"[162245, 251735, 258386, 141517, 157541]","[1506830731898, 1506830809579, 1506830868850, ...","[4, 4, 4, 4, 4]","[1, 1, 1, 1, 1]","[17, 17, 17, 17, 17]","[8, 8, 8, 8, 8]","[28, 28, 28, 28, 28]","[1, 1, 1, 1, 1]","[0.27982926537144653, 0.2840155957223325, 0.28...","[0.9600497811268313, 0.9588196605131168, 0.957...","[8.975979006501141e-07, 8.975979006501141e-07,...","[0.9999999999995972, 0.9999999999995972, 0.999...","[61.943305, 16.29960527777778, 17.648291666666...","[1.2589649905892517, 0.5028926468217809, 0.558..."
705,1506828968165442,705,1506828968000,2,"[119592, 284847]","[1506830912301, 1506830942301]","[4, 4]","[1, 1]","[17, 17]","[1, 1]","[21, 21]","[2, 2]","[0.29237195508272634, 0.2965418249991975]","[0.9563046794202694, 0.9